In [1]:
import pandas as pd

### Load data

In [2]:
test_boxes = pd.read_csv('tbl_test_boxes.csv')
train_boxes = pd.read_csv('tbl_train_boxes.csv')
unboxer_attributes = pd.read_csv('tbl_unboxer_attributes.csv')
unboxer_keywords = pd.read_csv('tbl_unboxer_keywords.csv')
vendor_keywords = pd.read_csv('tbl_vendor_keywords.csv')

C:\Users\britt\miniconda3\envs\bobcat_env_36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (61,63) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Look at the data 

In [3]:
def check_for_nans_by_column(df):
    for col in df.columns:
        n_nans = len(df[df[col].isna()==True])
        print('{}: {} nans'.format(col, n_nans))

In [4]:
# simple enough
train_boxes

,unboxer_id,vendor_id,target_bobcat
0,2116,53597,0
1,6430,17968,0
2,466,44148,0
3,3174,33479,0
4,8573,62492,1
...,...,...,...
116832,2961,17803,0
116833,4348,13982,0
116834,8749,40771,0
116835,219,52341,0


In [5]:
check_for_nans_by_column(train_boxes)

unboxer_id: 0 nans
vendor_id: 0 nans
target_bobcat: 0 nans


In [6]:
# same
test_boxes

,unboxer_id,vendor_id,target_bobcat
0,1,8324,NaN
1,1,10738,NaN
2,1,21305,NaN
3,1,45447,NaN
4,1,50490,NaN
...,...,...,...
116832,9421,66201,NaN
116833,9421,67055,NaN
116834,9421,68334,NaN
116835,9421,70180,NaN


In [7]:
check_for_nans_by_column(test_boxes)

unboxer_id: 0 nans
vendor_id: 0 nans
target_bobcat: 116837 nans


In [8]:
# will need to decide how to handle nans
# beartrap variables are categorical - will need to one-hot-encode

# it seems that skimask features can be 0, 1, or -1
# I assume these are meant to mean null, positive, or negative
# so then it would make sense to replace nulls with 0s
# then I could one hot encode 

unboxer_attributes

,unboxer_id,feature_skimask1,feature_skimask2,feature_skimask3,feature_skimask4,feature_skimask5,feature_skimask6,feature_skimask7,feature_skimask8,feature_skimask9,...,feature_skimask56,feature_skimask57,feature_skimask58,feature_skimask59,feature_skimask60,feature_beartrap1,feature_beartrap2,feature_beartrap3,feature_beartrap4,feature_beartrap5
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9,0,15
1,2,-1.0,1.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0,1.0,...,-1.0,-1.0,-1.0,-1.0,0.0,NaN,0.0,9,0,103
2,3,1.0,-1.0,-1.0,0.0,1.0,0.0,-1.0,0.0,0.0,...,-1.0,0.0,1.0,0.0,0.0,77,0.0,9,0,56
3,4,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,...,-1.0,1.0,1.0,-1.0,1.0,NaN,0.0,9,0,56
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9,0,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9022,9023,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,...,-1.0,1.0,1.0,0.0,1.0,NaN,0.0,7,0,94
9023,9024,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,0.0,...,-1.0,0.0,-1.0,0.0,0.0,NaN,0.0,9,0,100
9024,9025,NaN,NaN,NaN,NaN,-1.0,NaN,1.0,NaN,NaN,...,-1.0,NaN,NaN,NaN,NaN,NaN,0.0,9,0,22
9025,9026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,9,0,4


In [9]:
check_for_nans_by_column(unboxer_attributes)

unboxer_id: 0 nans
feature_skimask1: 6004 nans
feature_skimask2: 5199 nans
feature_skimask3: 5090 nans
feature_skimask4: 5971 nans
feature_skimask5: 4770 nans
feature_skimask6: 6109 nans
feature_skimask7: 4786 nans
feature_skimask8: 6088 nans
feature_skimask9: 5761 nans
feature_skimask10: 4667 nans
feature_skimask11: 6239 nans
feature_skimask12: 5443 nans
feature_skimask13: 5622 nans
feature_skimask14: 5525 nans
feature_skimask15: 6207 nans
feature_skimask16: 5788 nans
feature_skimask17: 4805 nans
feature_skimask18: 5486 nans
feature_skimask19: 4828 nans
feature_skimask20: 4979 nans
feature_skimask21: 5575 nans
feature_skimask22: 5830 nans
feature_skimask23: 4957 nans
feature_skimask24: 6297 nans
feature_skimask25: 5706 nans
feature_skimask26: 5132 nans
feature_skimask27: 5058 nans
feature_skimask28: 6191 nans
feature_skimask29: 5305 nans
feature_skimask30: 5414 nans
feature_skimask31: 5866 nans
feature_skimask32: 5455 nans
feature_skimask33: 5607 nans
feature_skimask34: 5278 nans
feat

In [10]:
# categorical - simple enough
# need to switch enabled to 0s and 1s
unboxer_keywords

,unboxer_id,keyword_id,enabled
0,9121,249,1
1,9089,249,1
2,9089,1963,1
3,9089,46,-1
4,9045,885,1
...,...,...,...
243056,1886,674,1
243057,4174,436,1
243058,4174,443,1
243059,4174,1056,1


In [11]:
check_for_nans_by_column(unboxer_keywords)

unboxer_id: 0 nans
keyword_id: 0 nans
enabled: 0 nans


In [12]:
# again, need to deal with nans - how many rows are all nan?
# need to switch enabled to 0s and 1s
vendor_keywords

,vendor_id,keyword_id,enabled
0,44975.0,244.0,1.0
1,44975.0,2117.0,1.0
2,44975.0,2063.0,1.0
3,44975.0,1417.0,1.0
4,44975.0,1620.0,1.0
...,...,...,...
1048570,NaN,NaN,NaN
1048571,NaN,NaN,NaN
1048572,NaN,NaN,NaN
1048573,NaN,NaN,NaN


In [13]:
# so it seems that we can just discard any rows with a nan
check_for_nans_by_column(vendor_keywords)

vendor_id: 614822 nans
keyword_id: 614822 nans
enabled: 614822 nans


### Clean the data 
Let's move backwards on this one

In [14]:
# drop rows with nans
vendor_keywords = vendor_keywords.dropna()

# replace -1s with 0s
vendor_keywords = vendor_keywords.replace(-1, 0)

In [15]:
vendor_keywords

,vendor_id,keyword_id,enabled
0,44975.0,244.0,1.0
1,44975.0,2117.0,1.0
2,44975.0,2063.0,1.0
3,44975.0,1417.0,1.0
4,44975.0,1620.0,1.0
...,...,...,...
433748,75092.0,1422.0,1.0
433749,75092.0,413.0,1.0
433750,75092.0,1964.0,1.0
433751,75092.0,501.0,1.0


In [16]:
# replace -1s with 0s
unboxer_keywords = unboxer_keywords.replace(-1, 0)

In [17]:
unboxer_keywords

,unboxer_id,keyword_id,enabled
0,9121,249,1
1,9089,249,1
2,9089,1963,1
3,9089,46,0
4,9045,885,1
...,...,...,...
243056,1886,674,1
243057,4174,436,1
243058,4174,443,1
243059,4174,1056,1


In [18]:
### replace nans with 0s 
unboxer_attributes = unboxer_attributes.fillna(0)

# now everything can be one-hot-encoded
# going to have so many columns... can definitely remove correlated/insignificant ones later if needed

# first we need to make all of the columns categorical
def make_all_cols_categorical(df):
    for col in df.columns:
        if col != 'unboxer_id':
            df[col] = df[col].astype('category')
    return(df)
unboxer_attributes = make_all_cols_categorical(unboxer_attributes)
# do the one hot encoding
unboxer_attributes = pd.get_dummies(unboxer_attributes)

In [19]:
# test_boxes and train_boxes are fine for now

### Combine the tables to make one big dataset